## **Advanced Data Visualization using hdf5 datafiles from Institut Néel**
- Version: 1.0.3c <br>
- Author: William Rigaut <br>
- Date: 19.08.2025  <br>
- Package developped at Institut Néel to visualize and manipulate the data with python commands. A tool (DaHU) to quickly visualize the data is already available at https://github.com/Combinatorials-Neel/DaHU. This Notebook will allow you to customize plots with much less restrictions but needs basic python knowledge.

### **1. Introduction**

<p>An <b>HDF5</b> datafile compiled from high-throughput experiments at Institut Néel contains different measurement scans for each point in a 2D-grid over a thin film deposited on a 100mm substrate. This 2D-grid contains a point every 5mm centered around the center of the wafer itself, by considering the circular shape of the substrate one datafile contains 249 scans for each main characterization technique.</p>

<p>At each position, four main characterization techniques can be performed: <p>

- <b>Energy Dispersive X-ray spectroscopy (EDX)</b> where element composition can be determined using a scanning electron microscope.

- <b>Scanning Magneto Optics Kerr Effect (MOKE)</b> where the coercive field on the surface of a magnetic thin film can be obtained by a fast optical measurement under a pulsed magnetic field.

- <b>X-Ray Diffraction (XRD)</b> where structural properties can be determined (Phases and lattice parameters) using either a automated laboratory diffractometer or synchrotron radiation.

- <b>Profilometry (DEKTAK)</b>  is also used to estimate magnetic film thickness at every position with a calibration experiment using a patterned substrate. This technique is new and it has not been applied to older datasets. Profilometry data will be added in a futur version of the package and is currently not supported.

The HDF5 file structure is defined using a standard very similar to the <b>NeXuS hdf5 file format</b> used in synchrotron facilities (https://www.nexusformat.org/).

<span style="color: red;">Disclamer: The Notebook was written as an example for a specific dataset and the example code will have to be slightly adapted for other datasets.</span>

### **2. Read and display data from hdf5 file using a GUI**

Before starting, you can download the example dataset [here](https://drive.google.com/drive/folders/1SQeCeuoVnfZD1pjYhtj-ZBLcgDnFYLYJ?usp=sharing). This file contains a full dataset for the (NdCe)FeB system.

A first overview of the HDF5 datafile can be done using a graphical user interface (GUI) program such as:
- `pyMca` http://www.silx.org/doc/PyMca/dev/install.html
- `hdf5view` https://pypi.org/project/hdf5view/

<p>Without GUI, the python module <b>h5py</b>, which is actually used by the reader can also be considered.</p>

<p>Here is an example with the PyMca software:</p>

![HDF5 file display using PyMca software](examples/Figure_1-PyMca.png)

Inside this hdf5 datafile example (NdCeFeB) you will be able to find at most 5 groups:
- `sample` containing basic sample info such as layer structure, deposition date, operator, ect...
- `NdCeFeB_1-1_EDX` containing all EDX scans at each wafer position.
- `NdCeFeB_2-5_ESRF` containing all XRD scans at each wafer position.
- `NdCeFeB_2-5_MOKE` containing all MOKE scans at each wafer position.

These group will be reviewed in more details using the reader itself, but feel free to explore the dataset on your own and please if you have any suggestions.

First, you need to install all the required packages using the following command:

In [ ]:
# This step is optional and not needed if all packages are already present 
#!source .venv/bin/activate
#!pip install -e .

### **3. Using the hdf5 reader from Institut Néel**

##### **3.1 Default xarray full dataset**

A built-in module has been written to parse the full hdf5 file, it will read all the data along with the attributs associated to each dataset and convert it to an xarray.Dataset object. 

From the xarrays you will be able to manipulate and plot the data as your convenience.

First, you can import the `read_hdf5` functions along the `pathlib` python library to use path that are OS independant.

In [ ]:
import neel_data_vis.readers.read_hdf5 as read_hdf5
import pathlib

Then you simply need to specify the path to your HDF5 datafile using the following command: 

In [2]:
HDF5_path = pathlib.Path("/Users/williamrigaut/Desktop/combinatorials/combinatorials-data/NdFeB_50mm.hdf5")
#HDF5_path = pathlib.Path("/Users/williamrigaut/Desktop/combinatorials/combinatorials-data/NdCeFeB_2-7.hdf5")

The package `read_hdf5` includes a function `get_full_dataset()` that will read inside the HDF5 file to retrieve results from <b>EDX</b>, <b>MOKE</b> and <b>XRD</b> for every position using an xarray.

You can call it by using the following:


In [3]:
data = read_hdf5.get_full_dataset(HDF5_path)
data # To display the xarray in the notebook

Data type EDX not found in HDF5 file.
Data type MOKE not found in HDF5 file.
Data type XRD not found in HDF5 file.


<xarray.Dataset> Size: 2kB
Dimensions:          (y: 16, x: 17)
Coordinates:
  * y                (y) float64 128B -40.0 -35.0 -30.0 -25.0 ... 25.0 30.0 35.0
  * x                (x) float64 136B -40.0 -35.0 -30.0 -25.0 ... 30.0 35.0 40.0
Data variables:
    measured_height  (y, x) float64 2kB nan nan nan nan nan ... nan nan nan nan

 An interactive widget will allow you to see the content of the dataset. Currently this function returns an xarray.Dataset object with Coordinates (x,y) and the Data variables are :
 
 - `A_i Composition` (in at.%) is the compofrsition of A_i from EDX quantification.
 
 - `coercivity_dmdh` is the derivate coercivity dM/dH.
 - `coercivity_m0` (in T) is the coercivity when M = 0.
 - `intercept_field` (in T) is the intercept field.
 - `max_kerr_rotation` (in °) is the maximum signal from MOKE.
 - `reflectivity` (in V) is the MOKE reflectivity.
 - `P_i Phase Fraction` (in wt.%) is the weight phase fraction of P_i from XRD.
 - `(P_i)_j Lattice Parameter j` (in nm) is the lattice parameter j of the phase P_i obtained from XRD.
 - `measured_height` (in nm) is the thickness of the film.

 Since the coordinate matrix is a 80x80mm square and the wafer sample is shape like a disk, some points are actually outside the wafer and does not have any values associated and are set to np.nan values instead.

If you are only interested in one data variable, for example `Coercivity`, you can simply select this data with:

In [81]:
coercivity = data["coercivity_m0"] # Data variables can be selected using the dictionary syntax in python
coercivity

<xarray.DataArray 'coercivity_m0' (y: 17, x: 17)> Size: 2kB
array([[       nan,        nan,        nan,        nan,        nan,
               nan, 0.69965305, 0.70649816, 0.69431504, 0.67916687,
        0.6414323 ,        nan,        nan,        nan,        nan,
               nan,        nan],
       [       nan,        nan,        nan,        nan, 0.7533019 ,
        0.8300083 , 0.81406389, 0.8356194 , 0.8537124 , 0.78731245,
        0.74788779, 0.67721641, 0.59555166,        nan,        nan,
               nan,        nan],
       [       nan,        nan,        nan, 0.80544614, 0.87498949,
        0.94460942, 1.01424921, 1.09287595, 1.06130136, 0.99110801,
        0.94623359, 0.70200093, 0.64792245, 0.56444716,        nan,
               nan,        nan],
       [       nan,        nan, 0.81293123, 0.92220567, 1.02803269,
        1.14114286, 1.2226827 , 1.26930991, 1.16160464, 1.20652779,
        1.05892928, 0.83714701, 0.74724782, 0.58872443, 0.50411627,
               nan,        nan],
       [       nan, 0.81592056, 0.87739012, 1.07464049, 1.20745967,
        1.31896897, 1.40422123, 1.41605322, 1.3414418 , 1.32049876,
        1.11769394, 1.01688621, 0.84509577, 0.70267961, 0.55512137,
        0.4646553 ,        nan],
...
       [       nan, 0.76362077, 0.83304236, 0.96174372, 0.99016773,
        0.9878612 , 1.05472562, 1.03144289, 0.92947883, 0.8957801 ,
        0.82783426, 0.71949423, 0.62876851, 0.50597844, 0.37675733,
        0.33016461,        nan],
       [       nan,        nan, 0.72854343, 0.76347288, 0.83379111,
        0.90261167, 0.87713424, 0.89102074, 0.89708291, 0.8032804 ,
        0.73471599, 0.60940636, 0.50380274, 0.44368801, 0.39706557,
               nan,        nan],
       [       nan,        nan,        nan, 0.65744362, 0.7299107 ,
        0.72430086, 0.72910374, 0.76677744, 0.74158778, 0.6914039 ,
        0.61671838, 0.50600954, 0.45370482, 0.34886057,        nan,
               nan,        nan],
       [       nan,        nan,        nan,        nan, 0.61017123,
        0.61114328, 0.57907165, 0.61067153, 0.56495717, 0.48333166,
        0.42150857, 0.35919667, 0.35378899,        nan,        nan,
               nan,        nan],
       [       nan,        nan,        nan,        nan,        nan,
               nan, 0.48440695, 0.49987449, 0.41930401, 0.35421203,
        0.34091646,        nan,        nan,        nan,        nan,
               nan,        nan]])
Coordinates:
  * y        (y) float64 136B -40.0 -35.0 -30.0 -25.0 ... 25.0 30.0 35.0 40.0
  * x        (x) float64 136B -40.0 -35.0 -30.0 -25.0 ... 25.0 30.0 35.0 40.0
Attributes:
    units:    T

Then if you want to only select a specific value for coercivity you can use the following:

In [82]:
coercivity.sel(x=0, y=20)

<xarray.DataArray 'coercivity_m0' ()> Size: 8B
array(0.92947883)
Coordinates:
    y        float64 8B 20.0
    x        float64 8B 0.0
Attributes:
    units:    T

In [83]:
# If needed you can convert from T to A/m using:
mu0 = 4 * 3.141592653589793 * 1e-7  # Vacuum permeability in T/(A/m)
Hc_in_T = coercivity.sel(x=0, y=20)  # coercivity in T
# Hc_in_T = 1.2  # coercivity in T
Hc_in_APerm = Hc_in_T/mu0
Hc_in_APerm.attrs['units'] = "A/m"
Hc_in_APerm

<xarray.DataArray 'coercivity_m0' ()> Size: 8B
array(739655.74937075)
Coordinates:
    y        float64 8B 20.0
    x        float64 8B 0.0
Attributes:
    units:    A/m

- To better visualize the data, you can plot the data in a 2D heatmap using `plot()` directly on the xarray. However for better consistancy with DaHU we choose to use the `plotly` library instead.

In [84]:
# Heatmap plot
import plotly.express as px

coercivity = data["coercivity_m0"] # Getting all the coercivity values

fig = px.imshow(coercivity, color_continuous_scale='rainbow') # using imshow from plotly
fig.update_layout(title="Coercivity Heatmap", xaxis_title="Position X (mm)", yaxis_title="Position Y (mm)", width=700, height=600) # Adding title, labels and figure 
fig.update_yaxes(autorange=True) # To have the y axis from -40 to 40

fig.show()

You can also select subsets with the `sel()` function by specifiying in argument which values of x and y you need. For more advanced xarray operations you can directly check https://docs.xarray.dev/en/stable/getting-started-guide/quick-overview.html

The following examples include a line selection for Nd composition.

In [85]:
# Line x=0 selected for Nd composition
import plotly.express as px

composition_Nd = data["Nd Composition"] # Nd composition data variable
line_composition = composition_Nd.sel(y=0) # Selecting the line y=0
coordinates = line_composition.coords["x"] # Change .coords["x"] to .coords["y"] if you plot at x fixed

fig = px.line(x=coordinates, y=line_composition, width=800, height=600) 
fig.update_layout(title="Nd Composition at line y=0", xaxis_title="Position X (mm)", yaxis_title="Nd Composition (at.%)")

fig.show()

Here is another heatmap example with the lattice parameter C for NdFeB :

In [86]:
# Lattice parameter C for NdFeB (Not present in the currently provided dataset, will give an error)
import plotly.express as px

lattice_parameter_C_NdFeB = data["Nd2Fe14B Lattice Parameter C"] #Lattice parameter C for NdFeB phase

fig = px.imshow(lattice_parameter_C_NdFeB, color_continuous_scale='rainbow')
fig.update_layout(title="Lattice parameter C for NdFeB phase", xaxis_title="Position X (mm)", yaxis_title="Position Y (mm)", width=800, height=600)
fig.update_yaxes(autorange=True)

fig.show()

You can then associate multiple datasets to plot for example lattice parameter C vs Ce over total RE content and thickness in a 1D plot. For this I recommand using the `pandas` module as well after reshaping our data:

In [121]:
# More manipulations
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

# Setting values for default NdFeB and CeFeB lattice parameters
DEFAULT_CE_CONTENT = [100, 0]
DEFAULT_LATTICE_PARAMETER_C = [1.211, 1.220]

# If we want to do operations on the data, we reshape it to a 1D array first
lattice_c = data["Nd2Fe14B Lattice Parameter C"].values.reshape(-1)
lattice_a = data["Nd2Fe14B Lattice Parameter A"].values.reshape(-1)
ce_content = data["Ce Composition"].values.reshape(-1)
fe_content = data["Fe Composition"].values.reshape(-1)
coercivity = data["coercivity_m0"].values.reshape(-1)
#Ce_alpha_fraction = data["Cerium Phase Fraction"].values.reshape(-1)
#thickness = data["measured_height"].values.reshape(-1)

# We can create a pandas dataframe with the reshaped 1D data, here we can directly do operations on 1D datasets if they have the same shape
df = pd.DataFrame({"Lattice Parameter A": lattice_a, "Lattice Parameter C": lattice_c, "Ce content": ce_content, "Coercivity (T)": coercivity, "Ce/RE content": 100*ce_content/(100-fe_content)})
# We can also include a mask to remove outliers, for example here we only want thickness values from 140 to 280
mask_min = 0.3; mask_max = 1.60
# mask = (df['Thickness'] > mask_min) & (df['Thickness'] < mask_max)
mask = (df['Coercivity (T)'] >= mask_min) & (df['Coercivity (T)'] <= mask_max)

# We create the plot with the scatter object in plotly, along with the default NdFeB and CeFeB lattice parameters with add_trace()
fig = px.scatter(df[mask], x= "Ce/RE content", y="Lattice Parameter C", color="Coercivity (T)", range_color=(mask_min, mask_max))
#fig = px.scatter(df, x= "Ce/RE content", y="Lattice Parameter C", color="Cerium Phase Fraction")
fig.add_trace(go.Scatter(x=DEFAULT_CE_CONTENT, y=DEFAULT_LATTICE_PARAMETER_C, marker={'color': 'blue'}, showlegend=False))

# Finally we update the layout with the titles
fig.update_layout(title="Lattice parameter C vs Ce content", xaxis_title="Ce/RE content (.%)", yaxis_title="Lattice Parameter C (nm)",
xaxis=dict(tickfont=dict(size=24), title_font=dict(size=28)), yaxis=dict(tickfont=dict(size=24), title_font=dict(size=28)), width=1250, height=800)
fig.update_traces(marker={'size': 10}) # To change the size of the markers
fig.update_coloraxes(colorbar_tickfont_size=24)
fig.update_xaxes(range=[20, 55]) # To modify the x axis range
fig.update_yaxes(range=[1.214, 1.219])

fig.show()

In [122]:
# More manipulations
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

# Setting values for default NdFeB and CeFeB lattice parameters
DEFAULT_CE_CONTENT = [100, 0]
DEFAULT_LATTICE_PARAMETER_A = [0.8760, 0.8800]

# If we want to do operations on the data, we reshape it to a 1D array first
lattice_c = data["Nd2Fe14B Lattice Parameter C"].values.reshape(-1)
lattice_a = data["Nd2Fe14B Lattice Parameter A"].values.reshape(-1)
ce_content = data["Ce Composition"].values.reshape(-1)
fe_content = data["Fe Composition"].values.reshape(-1)
coercivity = data["coercivity_m0"].values.reshape(-1)
#Ce_alpha_fraction = data["Cerium Phase Fraction"].values.reshape(-1)
#thickness = data["measured_height"].values.reshape(-1)

# We can create a pandas dataframe with the reshaped 1D data, here we can directly do operations on 1D datasets if they have the same shape
df = pd.DataFrame({"Lattice Parameter A": lattice_a, "Lattice Parameter C": lattice_c, "Ce content": ce_content, "Coercivity (T)": coercivity, "Ce/RE content": 100*ce_content/(100-fe_content), "Lattice Parameter C/A":lattice_c/lattice_a})
# We can also include a mask to remove outliers, for example here we only want thickness values from 140 to 280
mask_min = 0.3; mask_max = 1.60
# mask = (df['Thickness'] > mask_min) & (df['Thickness'] < mask_max)
mask = (df['Coercivity (T)'] >= mask_min) & (df['Coercivity (T)'] <= mask_max)

# We create the plot with the scatter object in plotly, along with the default NdFeB and CeFeB lattice parameters with add_trace()
fig = px.scatter(df[mask], x= "Ce/RE content", y="Lattice Parameter A", color="Coercivity (T)", range_color=(mask_min, mask_max))
#fig = px.scatter(df, x= "Ce/RE content", y="Lattice Parameter C", color="Cerium Phase Fraction")
fig.add_trace(go.Scatter(x=DEFAULT_CE_CONTENT, y=DEFAULT_LATTICE_PARAMETER_A, marker={'color': 'blue'}, showlegend=False))

# Finally we update the layout with the titles
fig.update_layout(title="Lattice parameter A vs Ce content", xaxis_title="Ce/RE content (.%)", yaxis_title="Lattice Parameter A (nm)",
xaxis=dict(tickfont=dict(size=24), title_font=dict(size=28)), yaxis=dict(tickfont=dict(size=24), title_font=dict(size=28)), width=1250, height=800)
fig.update_traces(marker={'size': 10}) # To change the size of the markers
fig.update_coloraxes(colorbar_tickfont_size=24)
fig.update_xaxes(range=[20, 60]) # To modify the x axis range
fig.update_yaxes(range=[0.876, 0.8795])

fig.show()

Another example is a ternary plot for Nd, Ce and Fe composition vs coercivity. Here we are also using `pandas` so the data is reshaped to a 1D array first. You can then zoom on the plot with plotly using the zoom button on the top right corner of the figure.

In [111]:
# Ternary plot
import plotly.express as px
import pandas as pd

# Rescaling of the composition to at.%
Nd = (data["Nd Composition"].values*100).reshape(-1)
Ce = (data["Ce Composition"].values*100).reshape(-1)
Fe = (data["Fe Composition"].values*100).reshape(-1)
coercivity = data["coercivity_m0"].values.reshape(-1)
lattice_c = data["Nd2Fe14B Lattice Parameter C"].values.reshape(-1)
lattice_a = data["Nd2Fe14B Lattice Parameter A"].values.reshape(-1)

# Creating the dataframe from the data
df = pd.DataFrame({"Nd Composition": Nd, "Ce Composition": Ce, "Fe Composition": Fe, "Coercivity": coercivity, "Lattice Parameter C": lattice_c, "Lattice Parameter A": lattice_a})

mask_min = 0.0; mask_max = 4.0
#mask_min = 0.0; mask_max = 2.5
mask = (df['Coercivity'] >= mask_min) & (df['Coercivity'] <= mask_max)

fig = px.scatter_ternary(df[mask], a='Nd Composition', b='Ce Composition', c='Fe Composition', color='Lattice Parameter C', color_continuous_scale='rainbow') # Using scatter_ternary from plotly
fig.update_layout(ternary_aaxis=dict(tickfont=dict(size=18), title_font=dict(size=20)),
ternary_baxis=dict(tickfont=dict(size=18), title_font=dict(size=20)),
ternary_caxis=dict(tickfont=dict(size=18), title_font=dict(size=20)),
width=1150, height=800)
fig.update_traces(marker={'size': 10})
fig.update_coloraxes(colorbar_tickfont_size=18)

fig.show()

##### **3.2 Getting xarray datasets containing measurement data**

If you need to work with the measurement readout instead, the function `get_measurement_data()` will return an xarray DataTree object containing all the scans of every experiment.

In [106]:
import neel_data_vis.readers.read_hdf5 as read_hdf5
import pathlib

#HDF5_path = pathlib.Path("/Users/williamrigaut/Desktop/combinatorials/combinatorials-data/CeCoZn_2-7.hdf5")
HDF5_path = pathlib.Path("/Users/williamrigaut/Desktop/combinatorials/combinatorials-data/HDF5/NdFeB_7-5.hdf5")

all_scans = read_hdf5.get_measurement_data(HDF5_path, datatype="all")

Reading EDX
Reading MOKE
Reading XRD


In [107]:
all_scans # You can visualize its content in the notebook by calling the variable

<xarray.DataTree 'Measurement Data'>
Group: /
├── Group: /EDX
│       Dimensions:  (y: 17, x: 17, counts: 4096, energy: 4096)
│       Coordinates:
│         * y        (y) float64 136B -40.0 -35.0 -30.0 -25.0 ... 25.0 30.0 35.0 40.0
│         * x        (x) float64 136B -40.0 -35.0 -30.0 -25.0 ... 25.0 30.0 35.0 40.0
│           counts   (y, x, counts) float64 9MB nan nan nan nan nan ... nan nan nan nan
│           energy   (y, x, energy) float64 9MB nan nan nan nan nan ... nan nan nan nan
├── Group: /MOKE
│       Dimensions:           (y: 19, x: 19, integrated_pulse: 2004,
│                              magnetization: 2004, pulse: 2004, reflectivity: 2004)
│       Coordinates:
│         * y                 (y) float64 152B -45.0 -40.0 -35.0 ... 35.0 40.0 45.0
│         * x                 (x) float64 152B -45.0 -40.0 -35.0 ... 35.0 40.0 45.0
│           integrated_pulse  (y, x, integrated_pulse) float64 6MB nan nan ... nan nan
│           magnetization     (y, x, magnetization) float64 6MB nan nan nan ... nan nan
│           pulse             (y, x, pulse) float64 6MB nan nan nan nan ... nan nan nan
│           reflectivity      (y, x, reflectivity) float64 6MB nan nan nan ... nan nan
└── Group: /XRD
        Dimensions:    (y: 17, x: 17, intensity: 2048, angle: 2048)
        Coordinates:
          * y          (y) float64 136B -40.0 -35.0 -30.0 -25.0 ... 25.0 30.0 35.0 40.0
          * x          (x) float64 136B -40.0 -35.0 -30.0 -25.0 ... 25.0 30.0 35.0 40.0
            intensity  (y, x, intensity) float64 5MB nan nan nan nan ... nan nan nan nan
            angle      (y, x, angle) float64 5MB nan nan nan nan nan ... nan nan nan nan

Before plotting, you need to select which dataset you want to plot as a function of another one, as an example :

In [108]:
# 1D pattern
import plotly.express as px

# Getting the angle and the intensity of the XRD pattern
angle = all_scans["XRD"]["angle"].sel(x=0, y=-40)
intensity = all_scans["XRD"]["intensity"].sel(x=0, y=-40)


fig = px.line(x=angle, y=intensity, width=800, height=600)
fig.update_layout(title="XRD Pattern at position x=0, y=-40", xaxis_title="Two Theta (°)", yaxis_title="Intensity (a.u.)")

fig.show()

One beautiful way to look at XRD patterns is with a waterfall plot. Here you select a line and you plot an image with the intensity on the color scale, the angle on the x axis and the position on the y axis.

In [109]:
# Waterfall pattern
import plotly.express as px

angle = all_scans["XRD"]["angle"].sel(x=0, y=0).values
intensity = all_scans["XRD"]["intensity"].sel(x=0).values
positions = all_scans["XRD"]["intensity"].sel(x=0).coords["y"] # Change to "y" if you plot at x fixed

fig = px.imshow(intensity, x=angle, y=positions, aspect="auto", color_continuous_scale='plasma')
fig.update_layout(title="XRD Waterfall Pattern at line x=0", xaxis_title="Two Theta (°)", yaxis_title="Position Y (mm)",
xaxis=dict(tickfont=dict(size=18), title_font=dict(size=20)),
yaxis=dict(tickfont=dict(size=18), title_font=dict(size=20)),
width=1300, height=900)

fig.update_coloraxes(colorbar_tickfont_size=20)
fig.update_yaxes(autorange=True)
#fig.update_xaxes(range=(1.6, 3.5))

fig.show()

In [ ]:
NdFeB_HDF5 = pathlib.Path("/Users/williamrigaut/Desktop/combinatorials/combinatorials-data/NdFeB_")
Dy_HDF5 = pathlib.Path("/Users/williamrigaut/Desktop/combinatorials/combinatorials-data/NdFeB_")

data_NdFeB = read_hdf5.get_full_dataset(NdFeB_HDF5)
data_Dy = read_hdf5.get_full_dataset(Dy_HDF5)

thickness_NdFeB = data_NdFeB["measured_height"]
thickness_Dy = data_Dy["measured_height"]

### **4. Data export to HDF5 and other formats**

##### **4.1 Simplified HDF5 dataset for Machine Learning**

A simplified HDF5 dataset can be created from a full HDF5 dataset using the function `create_simplified_dataset()`. This will create a new dataset with a structure that is different from the default format, making it easier to use for ML.

Part of the data will be copied and restructured as an hdf5 file containing 289 groups, one group for each position. In each group, datasets of different measurements will be directly created regardless of what instrument measured the data.

<span style="color: red;">Disclamer: The simplified HDF5 file can not be read by the current version of the reader since the structure is very different from the original file.</span>

In [85]:
import packages.readers.read_hdf5 as read_hdf5
import pathlib

# Filepath for the HDF5 files
foldername = None
# HDF5_path = pathlib.Path(f"/Users/williamrigaut/Desktop/combinatorials/combinatorials-data/{foldername}.hdf5")
HDF5_path = pathlib.Path("./datasets/NdCeFeB_2-5.hdf5")
#HDF5_simplified_path = pathlib.Path(f"/Users/williamrigaut/Desktop/combinatorials/combinatorials-data/{foldername}_simplified.hdf5")
HDF5_path = pathlib.Path("./datasets/NdCeFeB_2-5_simplified.hdf5")

# read_hdf5.create_simplified_dataset(HDF5_path, HDF5_simplified_path) # Will return the datatypes found inside the default hdf5 file


### **Appendix A. CHADA Tables and Diagrams for High Throughput Experiments at Néel Institut**

#### **A.1 CHADA Diagrams**

CHADA Diagrams are used to describe the flowchart of an experiment with a simple skematic. Each step is described with a short explanation and a color:

- RED: Indicates what type of sample was used to start the experiment.

- BLUE: Shows what needs to be done during the experiment to obtain the measured data.

- DARK GREEN: Raw data measured directly from the experiment.

- LIGHT GREEN: Treated data, usually extracted for the raw data using a software or a python code.

- DARK BLUE CONTOUR: Final treated data, is the content that you obtain from the `get_full_dataset()` command.

##### **CHADA Diagram for EDX (click left to display):**

![EDX Diagram](examples/EDX.svg)

##### **CHADA Diagram for MOKE:**

![MOKE Diagram](examples/MOKE.svg)

##### **CHADA Diagram for XRD:**

![XRD Diagram](examples/XRD.svg)

#### **A.2 CHADA Tables**
CHADA Tables are used to describe in details each experiment with a longer description that diagrams. Each important point about the experiment is described in a few lines. The color code is the same as for CHADA Tables.

##### **CHADA Table for EDX:**

![EDX Table](examples/CHADA_TABLES_EDX.svg)

##### **CHADA Table for MOKE:**

![MOKE Table](examples/CHADA_TABLES_MOKE.svg)

##### **CHADA Table for XRD:**

![XRD Table](examples/CHADA_TABLES_XRD.svg)

### **@end-of-notebook**